In [ ]:
"""
1. 베이스모델 이름
2. 허깅페이스 이름
3. 체크포인트 저장폴더 이름
4. 에폭기준 저장폴더 이름
"""

'\n1. 베이스모델 이름\n2. 허깅페이스 이름\n3. 체크포인트 저장폴더 이름\n4. 에폭기준 저장폴더 이름\n'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install datasets
!pip install trl
!pip install -U bitsandbytes

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from datasets import Dataset
import os, torch, json, wandb, subprocess
from sklearn.model_selection import train_test_split
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
)

import torch.nn as nn
from trl import SFTTrainer
import pandas as pd

import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BitsAndBytesConfig, pipeline

from tqdm.notebook import tqdm
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s e

In [3]:
base_model = "rtzr/ko-gemma-2-9b-it"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화 적용
    bnb_4bit_compute_dtype=torch.float16,  # 연산을 float16에서 수행
    bnb_4bit_use_double_quant=True,  # 추가적인 양자화 적용
)

tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          trust_remote_code = True
                                          )

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation='eager',
        quantization_config=quantization_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [4]:
train = pd.read_csv('/content/drive/MyDrive/dacon/open/train.csv', encoding = 'utf-8-sig')
train = train.drop(columns="ID")


In [5]:
def llama_formatting(examples):
  inputs = []
  outputs =[]
  for inp, out in zip(examples['input'], examples['output']):
    inputs.append(inp)
    outputs.append(out)
  return {"input" : inputs, "output": outputs}

llama_dataset = llama_formatting(train) #그냥 딕셔너리임.
llama_dataset_prompt_map = Dataset.from_dict(llama_dataset)

dataset = llama_dataset_prompt_map

EOS_TOKEN = tokenizer.eos_token
#AUG1x: 다음은 난독화된 한글 문장입니다. 원래 문장으로 복원해 주세요.
#AUG2x: 다음은 읽기 어렵게 변형된 한글 문장입니다. 모든 문장의 길이, 띄어쓰기, 특수문자는 그대로이며, 한글 글자만 바뀌었습니다. 원래 문장으로 복원해 주세요.
#AUG4x: 다음은 암호화된 한글 문장입니다. 단어의 길이는 그대로이고 한글 글자만 바뀌었습니다. 원래 문장으로 복원해 주세요.
def formatting_prompts_func(examples):
    instructions = examples["input"]
    outputs = examples["output"]
    texts = []

    alpaca_prompt = """<bos><start_of_turn>user
    다음은 난독화된 한글 문장입니다. 원래 문장으로 복원해 주세요.

    변형된 문장: {}
    <end_of_turn>
    <start_of_turn>model
    원래 문장: {}<end_of_turn><eos>"""

    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output)  # .format() 사용
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/11263 [00:00<?, ? examples/s]

In [6]:
split_dataset = dataset.train_test_split(test_size=0.01, seed=42)
split_dataset
train_dataset = split_dataset["train"]
valid_dataset = split_dataset["test"]

In [7]:


# def find_all_linear_names(model):
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, nn.Linear):
#             names = name.split('.')
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])
#     if 'lm_head' in lora_module_names:
#         lora_module_names.remove('lm_head')
#     return list(lora_module_names)

# modules = find_all_linear_names(model)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)

# model, tokenizer = setup_chat_format(model, tokenizer)
# LoRA 모델 생성
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 모델을 명시적으로 훈련 모드로 설정
model.train()

trainable params: 54,018,048 || all params: 9,295,724,032 || trainable%: 0.5811


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
        

In [12]:
import wandb

wandb.init(
    project="[DACON] Decryption",
    name="saltware/sapie-gemma2-9B-it_AUG:1x",
    )

tokenizer.padding_side = "right"

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=train_dataset,  # 학습 데이터셋
    eval_dataset=valid_dataset,
    # dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    # max_seq_length=4096,  # 최대 시퀀스 길이
    # packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=1,  # 각 디바이스당 훈련 배치 크기
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,  # 그래디언트 누적 단계
        warmup_steps=10,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        # max_steps=CFG['STEPS'],  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        eval_steps=500,
        logging_steps=1,  # logging 스텝 수
        learning_rate=3e-4,  # 학습률
        fp16= False,  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16= True,  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="paged_adamw_32bit",
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=42,  # 랜덤 시드
        output_dir="/content/drive/MyDrive/dacon/ko-gemma-2-9b-it_AUG:1x_EPOCH:3",
        report_to="wandb",
    ),
)

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sungcoook (sungcook) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-60111fed9e57>:11: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/11150 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/11150 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/11150 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/113 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/113 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/113 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/113 [00:00<?, ? examples/s]

In [13]:
model.config.use_cache = False# True? ->
# model.gradient_checkpointing_enable()
trainer.train()
# seed42로 증강한 데이터셋으로 학습, 학습률 3e-4 --> 1e-4로 변경

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# save_dir = "/content/drive/MyDrive/dacon/ko-gemma-2-9b-it_AUG:2x/EPOCH-3"
# trainer.model.save_pretrained(save_dir)

In [ ]:
# #모델 합병하는 코드
# save_dir = "/content/drive/MyDrive/dacon/ko-gemma-2-9b-it_AUG:1x_EPOCH:3/checkpoint-3000"


# base_model = "rtzr/ko-gemma-2-9b-it"
# FINETUNE_MODEL = save_dir

# model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto', torch_dtype=torch.float16)
# model = PeftModel.from_pretrained(model, FINETUNE_MODEL, device_map='auto', torch_dtype=torch.float16)

# model = model.merge_and_unload()
# model.save_pretrained('/content/drive/MyDrive/dacon/ko-gemma-2-9b-it_AUG:1x_EPOCH:3/checkpoint-3000_merged')